In [ ]:
%%capture
# Passo a passo para executar o treinamento
# 1. Baixar as imagens do roboflow
# 2. Executar o slice_images na pasta MELHORE_IMAGES
# 3. Depois pegar o dataset e upar aqui
# 4. Apos a instalacao do sahi, ir até o diretorio de slicing.py que fica em:/usr/local/lib/python3.11/dist-packages/sahi/slicing.py e trocar a linha 141 por isso
# sliced_coco_annotation_list: List[CocoAnnotation] = []
# for coco_annotation in coco_annotation_list:
#     try:
#         if annotation_inside_slice(coco_annotation.json, slice_bbox):
#             sliced_coco_annotation = coco_annotation.get_sliced_coco_annotation(slice_bbox)
#             if sliced_coco_annotation.area / coco_annotation.area >= min_area_ratio:
#                 sliced_coco_annotation_list.append(sliced_coco_annotation)
#     except:
#         pass
#         # print("Não divisivel por 0 " + str(coco_annotation))
# return sliced_coco_annotation_list
# 5. Adicionar o arquivo data.yaml abaixo
# train: ../train/images
# val: ../valid/images
# test: ../test/images

# nc: 2
# names: ['black_point', 'vigia']

# roboflow:
#   workspace: computervisionfutebol
#   project: glass-blur
#   version: 20
#   license: CC BY 4.0
#   url: https://universe.roboflow.com/computervisionfutebol/glass-blur/dataset/20


#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install -U ultralytics
!pip install roboflow



import warnings
import os
import json
import shutil


warnings.simplefilter("ignore")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="T2pa1zR5aO10PH2kjkgu")
project = rf.workspace("computervisionfutebol").project("cardboard-box-detection-mozxg")
version = project.version(7)
dataset = version.download("yolov12")


dataset_version = version.name.replace(" ", "-") + "-" + version.version
dataset_version


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Cardboard-Box-Detection-7 in yolov12:: 100%|██████████| 8418/8418 [00:02<00:00, 2967.29it/s]


'Cardboard-Box-Detection-7'

In [ ]:
#%%writefile {dataset_version}/data.yaml
# train: ../train/images
# val: ../valid/images
# test: ../test/images

# nc: 2
# names: ["metragem", "plates"]

# roboflow:
#   workspace: computervisionfutebol
#   project: plates_detection
#   version: 22
#   license: CC BY 4.0
#   url: https://universe.roboflow.com/computervisionfutebol/glass-blur/dataset/20


In [ ]:
# import gdown
# file_id = "1rQSMWMmlk38NXbGph_pAEFhLJoJ0gM12"
# output_file = "dataset.rar"
# gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

# TREINA O MODELO

In [ ]:
from ultralytics import YOLO
import torch
import os

os.environ['WANDB_MODE'] = 'disabled'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = torch.device("cuda")
model = YOLO('yolo11s.pt').to(device)
#model = YOLO('yolov8s-p2.yaml').load('yolov8s.pt')

# Training.
results = model.train(
   #data='../content/DATASET/data.yaml',
   task="detect",
   data=f'/content/{dataset_version}/data.yaml',
   imgsz=640,
   epochs=5,
   batch=150,
   seed=42,
   exist_ok=True,
   name='model',
   pretrained=True
)

# Map 0.000336
# 0.00218

In [ ]:
# import localew
# locale.getpreferredencoding = lambda: "UTF-8"

# !zip -r runs.zip /content/runs/

In [ ]:
from google.colab import files
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Compactar a pasta /content/runs
!zip -r runs.zip /content/runs/

# Fazer o download do arquivo ZIP
files.download("runs.zip")